In [2]:
# !pip install papermill ipywidgets jupyter-client==6.1.5

     |################################| 107 kB 8.2 MB/s eta 0:00:01
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.3
    Uninstalling jupyter-client-6.1.3:
      Successfully uninstalled jupyter-client-6.1.3


In [1]:
import papermill as pm
import multiprocessing as mp
from sklearn.model_selection import ParameterGrid
import numpy as np
import os
import nest_asyncio
nest_asyncio.apply()

In [2]:
max_experiments = 5
input_notebook = 'XGBoost_Demo.ipynb'
output_dir = 'xgboost_2020_08_14'
hyperparameter_ranges = {
    'max_depth'    : [int(i) for i in np.random.randint(5, 15+1, 5)],
    'num_rounds' : [int(i) for i in np.random.randint(10, 50+1, 5)],
    'eta' : [float(i) for i in np.random.uniform(0.1, 1.0, 5)]
} 

In [3]:
candidate_params = np.random.choice(ParameterGrid(hyperparameter_ranges), max_experiments)

print(candidate_params[0])

{'eta': 0.599051300182406, 'max_depth': 14, 'num_rounds': 35}


In [4]:
def run_papermill(cand, inp, outd):
    
    if not os.path.exists(outd):
        os.makedirs(outd)
    
    output_notebook = outd + '/' + inp.split('.')[0] + str(cand['max_depth']) + '_' + str(cand['num_rounds']) + '_' + str(int(cand['eta']*100)) + '.ipynb'
    
    if os.path.exists(output_notebook):
        os.remove(output_notebook)
    
    print(cand, "starting")
    
    pm.execute_notebook(inp, output_notebook, dict(max_depth=cand['max_depth'], eta = cand['eta'], num_rounds=cand['num_rounds']))
    
    print(output_notebook, "completed")

In [5]:
for candidate in candidate_params:
    p = mp.Process(
        target=run_papermill,
        args=(candidate, input_notebook, output_dir,)
    )
    p.start()

{'eta': 0.599051300182406, 'max_depth': 14, 'num_rounds': 35} starting
{'eta': 0.599051300182406, 'max_depth': 9, 'num_rounds': 32} starting 
{'eta': 0.599051300182406, 'max_depth': 14, 'num_rounds': 40}starting
{'eta': 0.599051300182406, 'max_depth': 6, 'num_rounds': 40} starting
{'eta': 0.6944526246344632, 'max_depth': 9, 'num_rounds': 40} starting


/opt/conda/envs/rapids/lib/python3.6/site-packages/jupyter_client/manager.py:358: FutureWarning: Method cleanup(connection_file=True) is deprecated, use cleanup_resources(restart=False).
  FutureWarning)



xgboost_2020_08_14/XGBoost_Demo6_40_59.ipynb completed


/opt/conda/envs/rapids/lib/python3.6/site-packages/jupyter_client/manager.py:358: FutureWarning: Method cleanup(connection_file=True) is deprecated, use cleanup_resources(restart=False).
  FutureWarning)



xgboost_2020_08_14/XGBoost_Demo9_32_59.ipynb completed


...


xgboost_2020_08_14/XGBoost_Demo9_40_69.ipynb completed


/opt/conda/envs/rapids/lib/python3.6/site-packages/jupyter_client/manager.py:358: FutureWarning: Method cleanup(connection_file=True) is deprecated, use cleanup_resources(restart=False).
  FutureWarning)



xgboost_2020_08_14/XGBoost_Demo14_35_59.ipynb completed


/opt/conda/envs/rapids/lib/python3.6/site-packages/jupyter_client/manager.py:358: FutureWarning: Method cleanup(connection_file=True) is deprecated, use cleanup_resources(restart=False).
  FutureWarning)



xgboost_2020_08_14/XGBoost_Demo14_40_59.ipynb completed


In [7]:
# !LC_ALL=C.UTF-8 LANG=C.UTF-8 papermill XGBoost_Demo.ipynb XGBoost_Demo4.ipynb -p max_depth 13 -p eta 0.19026762501776856 -p num_rounds 22

Input Notebook:  XGBoost_Demo.ipynb
Output Notebook: XGBoost_Demo4.ipynb
Generating grammar tables from /opt/conda/envs/rapids/lib/python3.6/site-packages/blib2to3/Grammar.txt
Writing grammar tables to /root/.cache/black/19.10b0/Grammar3.6.10.final.0.pickle
Writing failed: [Errno 2] No such file or directory: '/root/.cache/black/19.10b0/tmpcthuy_go'
Generating grammar tables from /opt/conda/envs/rapids/lib/python3.6/site-packages/blib2to3/PatternGrammar.txt
Writing grammar tables to /root/.cache/black/19.10b0/PatternGrammar3.6.10.final.0.pickle
Writing failed: [Errno 2] No such file or directory: '/root/.cache/black/19.10b0/tmpp39tb2yq'
Executing:  96%|��������������������������������������������������������������������������������������� | 22/23 [00:09<00:00,  1.35cell/s]/opt/conda/envs/rapids/lib/python3.6/site-packages/jupyter_client/manager.py:358: FutureWarning: Method cleanup(connection_file=True) is deprecated, use cleanup_resources(restart=False).
  FutureWarning)
Executing: 10